In [2]:
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
m = Basemap(projection='robin', lon_0=0, resolution='l')
lats = [24.48237852, 26.89169118]
lons = [118.1558955, 117.1760012]
m.drawcountries(color='#ffffff', linewidth=0.5)
m.fillcontinents(color='#c0c0c0', lake_color='#ffffff')
x, y = m(lons, lats)
plt.plot(x, y, 'bo', color='r', markersize=5)
plt.show()


<class 'ModuleNotFoundError'>: No module named 'mpl_toolkits.basemap'